In [21]:
from pathlib import Path

import pyrosm
import geopandas as gpd
import pandas as pd
import shapely
import pyproj

import src.data.mapillary as mapi
import src.data.osm as osmd
#from src.data.mapillary import download_mapillary_image_by_key, download_mapillary_object_detection_by_key
#from src.data.osm import add_mapillary_key_to_network
from ipyleaflet import Map, Marker, Icon, Polyline, FullScreenControl
from importlib import reload
from tqdm.notebook import tqdm

### Settings

In [106]:
input_filepath = "../data/raw/berlin-latest.osm.pbf"
min_quality_score = 4
output_dir = Path("../data/processed/out")

### Code (see make_dataset.py)

In [5]:
osm = pyrosm.OSM(input_filepath)

In [6]:
network = osm.get_network(
    network_type="cycling",
    extra_attributes=["surface", "smoothness"]
)

In [10]:
network = network.head(100)

In [7]:
# Filter relevant columns
network = network[["id", "geometry", "surface", "smoothness"]]

In [8]:
# Filter only records where both surface and smoothness is set
network = network[(~network["surface"].isna()) & (~network["smoothness"].isna())]

In [102]:
# Get Mapillary keys for each street
street_mapillary_df = osmd.add_mapillary_key_to_network(network.head(1), 
                                                        street_buffer=5,
                                                        shorten_street_by=30,
                                                        min_quality_score=min_quality_score)

Request URL: https://a.mapillary.com/v3/images?client_id=TmxURHBKRFFFRTc1dWZscGpLWW5VUDo1ZjYyYmFjYjY5MTA3MDNk&bbox=13.526105139690427,52.4891476086417,13.5277082950903,52.49031200789549&per_page=500&sort_by=key&min_quality_score=4
DONE
Total images: 42


In [103]:
street_mapillary_df

,geometry,id,img_coords,mapillary_key,smoothness,surface
2,"MULTILINESTRING ((13.52783 52.49043, 13.52639 ...",4045220.0,POINT (13.5268612 52.4896728),DQYk9CKOsaVKI5jgJyXsnQ,bad,sett
2,"MULTILINESTRING ((13.52783 52.49043, 13.52639 ...",4045220.0,POINT (13.5268964 52.4896935),dfHVHCxaiLxOq5kxh88NeA,bad,sett
2,"MULTILINESTRING ((13.52783 52.49043, 13.52639 ...",4045220.0,POINT (13.5269718 52.4897393),zyibnfVuppLVnlgGTImXxQ,bad,sett
2,"MULTILINESTRING ((13.52783 52.49043, 13.52639 ...",4045220.0,POINT (13.5270097 52.4897627),pL2n6QmeQSa0mVlTyoLilw,bad,sett
2,"MULTILINESTRING ((13.52783 52.49043, 13.52639 ...",4045220.0,POINT (13.5270929 52.4898373),rmpE7Tg1R2Vrb3W6ET2dQA,bad,sett
2,"MULTILINESTRING ((13.52783 52.49043, 13.52639 ...",4045220.0,POINT (13.5271284 52.4898888),n6urwZz0q3NBs18aD3NULQ,bad,sett
2,"MULTILINESTRING ((13.52783 52.49043, 13.52639 ...",4045220.0,POINT (13.5271522 52.4899155),NdcqcMPINtZwd_LfOozWdw,bad,sett
2,"MULTILINESTRING ((13.52783 52.49043, 13.52639 ...",4045220.0,POINT (13.5274044 52.4901188),apY4zo5qfDD83lOJjdJEpQ,bad,sett
2,"MULTILINESTRING ((13.52783 52.49043, 13.52639 ...",4045220.0,POINT (13.5274312 52.490138),zddDUI090ORSkTD8d2-SXA,bad,sett


In [11]:
# Create output dir
Path(f"{output_dir}").mkdir(parents=True, exist_ok=True)

In [108]:
# Create sub dirs
image_dir = output_dir/ "images"
object_detection_dir = output_dir / "object_detections"

image_dir.mkdir(exist_ok=True)
object_detection_dir.mkdir(exist_ok=True)

In [109]:
# Export street_mapillary_df as data.csv
csv_filepath = output_dir / "data.csv"
street_mapillary_df.to_csv(csv_filepath, index=False)

In [77]:
# Download images and object detections
for _, row in tqdm(street_mapillary_df.iterrows(), total=len(street_mapillary_df)):
    mapi.download_mapillary_image_by_key(row["mapillary_key"], download_dir=image_dir)
    #download_mapillary_object_detection_by_key(row["mapillary_key"],
                                               #download_dir=object_detection_dir)

  0%|          | 0/1 [00:00<?, ?it/s]

../data/processed/out/images/pL2n6QmeQSa0mVlTyoLilw.jpg already exists. Skipping Download.


In [104]:
# Display a map with the streets and images

map_center = street_mapillary_df.iloc[0].img_coords
map_center = [map_center.y, map_center.x]

street_id_added = list()  # list with added streets to avoid adding them multiple times
m = Map(zoom=16, center=map_center, max_zoom=22, scroll_wheel_zoom=True)
for _, row in street_mapillary_df.iterrows():
    img_path = Path(output_dir) / "images" / (row.mapillary_key + ".jpg")
    assert img_path.is_file(), img_path
    icon = Icon(icon_url=str(img_path), 
                icon_size=[177, 100])
    marker_center = [row.img_coords.y, row.img_coords.x]
    marker = Marker(location=marker_center, draggable=False, icon=icon)
    m.add_layer(marker)
    
    if row.id not in street_id_added:
        street_id_added.append(row.id)
        line_shapely = row.geometry
        line_map_locactions = list()

        for single_line in line_shapely.geoms:
            line_map_locactions.extend([reversed(p) for p in single_line.coords])
        
        line = Polyline(
            locations=line_map_locactions,
            color="green" ,
            fill=False
        )
        m.add_layer(line)
        
m.add_control(FullScreenControl())       
m

Map(center=[52.4896728, 13.5268612], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…